# LangChain Demonstration Notebook
This notebook demonstrates how to:
- Load a text file.
- Split it into manageable chunks.
- Generate embeddings using OpenAI.
- Use FAISS for vector storage and retrieval.
- Set up a question-answering system with LangChain.

In [ ]:
# Install necessary packages
%pip install langchain langchain-community faiss-cpu

In [11]:
# Import required libraries
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [12]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2:latest")


### Step 2: Load the text file
The text file is loaded using `TextLoader`. Replace the file path with your desired text file.

In [ ]:
# Load the text file
file_path = "../0-Data/paul_graham_essay3.txt"  # Replace with your text file path
loader = TextLoader(file_path)
documents = loader.load()
print(documents[0])  # Display the first document to verify loading 

### Step 3: Split the text into manageable chunks
To handle large documents, the text is split into smaller chunks with overlap for better context.

In [ ]:
# Split the text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

### Step 4: Generate embeddings for the text chunks
Embeddings are generated for each chunk using OpenAI's embeddings model.

In [15]:
# Generate embeddings for text chunks
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2:latest",
)

vector_store = FAISS.from_documents(split_docs, embeddings)

### Step 5: Set up a retrieval-based QA system
The system retrieves relevant chunks and uses OpenAI's model to answer queries.

In [16]:
# Set up a retrieval-based QA system
retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

### Step 6: Ask questions
A loop is provided to allow users to ask questions interactively. Type `exit` to quit.

In [ ]:
# Interactive QA loop
while True:
    query = input("\nEnter your question (or 'exit' to quit): ")
    if query.lower() == "exit":
        print("Exiting...")
        break
    result = qa_chain.invoke({"query": query})
    print("\nAnswer:")
    print(result["result"])